In [1]:
!pip install -qU crewai langchain_openai 'crewai[tools]' google-search-results

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.5/634.5 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

###Initialize Models

In [4]:
from langchain_openai import ChatOpenAI
from crewai import LLM

Gemini = LLM(
    model="gemini/gemini-2.0-flash",)


#gpt4o = ChatOpenAI(model = "gpt-4o")

### Stock Market Agent 🧳


In [5]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# Initialize tools
search_tool = SerperDevTool()

def create_stock_market_agent(stock_symbol):
    # Define the Stock Market Agent
    stock_agent = Agent(
        role="Financial Analyst",
        goal=f"Analyze the stock market for {stock_symbol}",
        backstory="A financial expert trained to analyze markets and provide actionable investment insights",
        verbose=True,
        tools=[search_tool],
        llm=Gemini
    )

    # Define the Market Trend Analyst
    trend_analyst = Agent(
        role="Market Trend Analyst",
        goal=f"Analyze broader market trends affecting {stock_symbol}",
        backstory="A seasoned market analyst specializing in sector analysis, market trends, and competitive landscape evaluation",
        verbose=True,
        tools=[search_tool],
        llm=Gemini
    )

    # Define the Stock Analysis Task
    stock_task = Task(
        description=f"Analyze the performance and trends of {stock_symbol} and provide investment recommendations",
        expected_output=f"A detailed analysis of {stock_symbol} including trends, risks, and recommendations.",
        agent=stock_agent
    )

    # Define the Market Analysis Task
    market_task = Task(
        description=f"Analyze sector trends, market conditions, and competitive landscape affecting {stock_symbol}. Include macroeconomic factors and sector-specific catalysts.",
        expected_output=f"A comprehensive market analysis including sector trends, competitive position, and market factors affecting {stock_symbol}.",
        agent=trend_analyst
    )

    # Create and Run the Crew
    crew = Crew(
        agents=[stock_agent, trend_analyst],
        tasks=[stock_task, market_task],
        verbose=True,
        process=Process.sequential
    )

    result = crew.kickoff(inputs={"stock_symbol": stock_symbol})
    return result

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


### Execute the agent


In [6]:
stock_symbol = input("Enter the stock symbol (e.g., AAPL, TSLA): ")
stock_result = create_stock_market_agent(stock_symbol)
print("Stock Market Analysis:")
print(stock_result)

Enter the stock symbol (e.g., AAPL, TSLA): APPL


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5c70c1fd-1836-41ec-9da2-cad30050f388                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Financial Analyst                                                                                       │
│                                                                                                                 │
│  Task: Analyze the performance and trends of APPL and provide investment recommendations                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Trend Analyst                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Apple (AAPL) Stock Analysis and Investment Recommendations                                                  │
│                                                                                                                 │
│  **Executive Summary:**                                                                                         │
│                                                                                                                 │
│  Apple (AAPL) is a dominant player in the technology sector, known for its iconic products and loyal customer   │
│  base. The stock currently trades around $210-$211, and analyst consensus points to a "Buy" rating with an      │
│  average price target in the $235-$237 range, suggesting a potential upside of 12-13%. However, a wide range    │
│  of price targets indicates significant uncertainty. While Apple possesses strong fundamentals, investors       │
│  should carefully consider several risks and challenges before investing. According to 34 analysts, the         │
│  average rating for AAPL stock is "Buy" with a 12-month stock price target of $237.36, an increase of 11.19%    │
│  from the latest. Some analysts estimate a high of $275.00 and a low of $139.00 for AAPL price.                 │
│                                                                                                                 │
│  **Trends:**                                                                                                    │
│                                                                                                                 │
│  *   **Growth in Services:** Apple's services segment (including Apple Music, iCloud, and the App Store)        │
│  continues to be a significant growth driver, providing recurring revenue and diversification.                  │
│  *   **Wearables and Accessories:** The wearables and accessories category (including AirPods and Apple Watch)  │
│  is also experiencing strong growth.                                                                            │
│  *   **Expansion into New Markets:** Apple is exploring new markets, such as augmented reality (AR) and         │
│  virtual reality (VR), which could present future growth opportunities.                                         │
│  *   **Share Repurchases and Dividends:** Apple has a history of returning capital to shareholders through      │
│  share repurchases and dividends.                                                                               │
│  *   **Focus on Sustainability:** Apple is increasingly focused on sustainability initiatives, which could      │
│  appeal to environmentally conscious investors.                                                                 │
│  *   **Revenue Growth**: Apple reported year-on-year revenue growth of 5.1% this past quarter, exceeding        │
│  estimates by 0.7%. The market expects continued growth fueled by trends like remote work infrastructure and    │
│  sustainable practices, with the market potentially reaching $914.55 billion by 2028 at a 6.5% CAGR.            │
│                                                                                                                 │
│  **Risks and Challenges:**                                                                                      │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 52f54f3a-b7d6-476f-b316-4355cf1c10ed                                                                     │
│  Agent: Market Trend Analyst                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5c70c1fd-1836-41ec-9da2-cad30050f388                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Apple (AAPL) Stock Analysis and Investment Recommendations                                    │
│                                                                                                                 │
│  **Executive Summary:**                                                                                         │
│                                                                                                                 │
│  Apple (AAPL) is a dominant player in the technology sector, known for its iconic products and loyal customer   │
│  base. The stock currently trades around $210-$211, and analyst consensus points to a "Buy" rating with an      │
│  average price target in the $235-$237 range, suggesting a potential upside of 12-13%. However, a wide range    │
│  of price targets indicates significant uncertainty. While Apple possesses strong fundamentals, investors       │
│  should carefully consider several risks and challenges before investing. According to 34 analysts, the         │
│  average rating for AAPL stock is "Buy" with a 12-month stock price target of $237.36, an increase of 11.19%    │
│  from the latest. Some analysts estimate a high of $275.00 and a low of $139.00 for AAPL price.                 │
│                                                                                                                 │
│  **Trends:**                                                                                                    │
│                                                                                                                 │
│  *   **Growth in Services:** Apple's services segment (including Apple Music, iCloud, and the App Store)        │
│  continues to be a significant growth driver, providing recurring revenue and diversification.                  │
│  *   **Wearables and Accessories:** The wearables and accessories category (including AirPods and Apple Watch)  │
│  is also experiencing strong growth.                                                                            │
│  *   **Expansion into New Markets:** Apple is exploring new markets, such as augmented reality (AR) and         │
│  virtual reality (VR), which could present future growth opportunities.                                         │
│  *   **Share Repurchases and Dividends:** Apple has a history of returning capital to shareholders through      │
│  share repurchases and dividends.                                                                               │
│  *   **Focus on Sustainability:** Apple is increasingly focused on sustainability initiatives, which could      │
│  appeal to environmentally conscious investors.                                                                 │
│  *   **Revenue Growth**: Apple reported year-on-year revenue growth of 5.1% this past quarter, exceeding        │
│  estimates by 0.7%. The market expects continued growth fueled by trends like remote work infrastructure and    │
│  sustainable practices, with the market potentially reaching $914.55 billion by 2028 at a 6.5% CAGR.            │
│                                                                                                                 │
│  **Risks and Challenges:**                            

Stock Market Analysis:
## Apple (AAPL) Stock Analysis and Investment Recommendations

**Executive Summary:**

Apple (AAPL) is a dominant player in the technology sector, known for its iconic products and loyal customer base. The stock currently trades around $210-$211, and analyst consensus points to a "Buy" rating with an average price target in the $235-$237 range, suggesting a potential upside of 12-13%. However, a wide range of price targets indicates significant uncertainty. While Apple possesses strong fundamentals, investors should carefully consider several risks and challenges before investing. According to 34 analysts, the average rating for AAPL stock is "Buy" with a 12-month stock price target of $237.36, an increase of 11.19% from the latest. Some analysts estimate a high of $275.00 and a low of $139.00 for AAPL price.

**Trends:**

*   **Growth in Services:** Apple's services segment (including Apple Music, iCloud, and the App Store) continues to be a significant growth d